In [1]:
!pip install transformers torch numpy datasets accelerate

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
import logging
from collections import defaultdict
from sklearn.preprocessing import MinMaxScaler
import os
import random

# Set environment variable for memory management
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Set up logging
logging.basicConfig(filename='gesal_training_logs.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())  # Log to console

# Custom Linear Layer with SVF
class SVFLinear(nn.Module):
    def __init__(self, original_linear):
        super().__init__()
        self.original_linear = original_linear
        with torch.no_grad():
            U, Sigma, V = torch.svd(original_linear.weight.float())
            self.U = nn.Parameter(U, requires_grad=False)
            self.Sigma = nn.Parameter(Sigma, requires_grad=False)
            self.V = nn.Parameter(V.t(), requires_grad=False)
        self.z = nn.Parameter(torch.ones_like(self.Sigma), requires_grad=True)

    def forward(self, x):
        Sigma_z = self.Sigma * self.z
        Vx = torch.matmul(self.V, x.T if x.dim() == 2 else x.unsqueeze(-1))
        Sigma_Vx = Sigma_z.unsqueeze(-1) * Vx
        output = torch.matmul(self.U, Sigma_Vx)
        if self.original_linear.bias is not None:
            output = output + self.original_linear.bias.unsqueeze(-1)
        return output.squeeze(-1) if x.dim() == 2 else output

# Graph Node for Task-Specific Adaptations
class Node:
    def __init__(self, embedding, z_vectors, count=1):
        self.embedding = embedding
        self.z_vectors = z_vectors
        self.count = count
        self.past_responses = set()

    def update_embedding(self, new_embedding):
        self.embedding = (self.count * self.embedding + new_embedding) / (self.count + 1)
        self.count += 1

# Self-Adaptive LLM with Graph Structure
class AdaptiveLLM:
    def __init__(self, model_name="meta-llama/llama-3.2-1b-instruct", distance_threshold=0.1, buffer_size=20, temperature=0.3, top_k=20, lr=0.005):
        from huggingface_hub import login
        login(token="XXXXXXXXXXXXXX")  # Replace with your token

        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id
        self.base_model = AutoModelForCausalLM.from_pretrained(model_name)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.base_model.to(self.device)

        self.svf_layers = []
        for name, module in self.base_model.named_modules():
            if isinstance(module, nn.Linear) and "mlp" in name:
                svf_layer = SVFLinear(module)
                self.svf_layers.append(svf_layer)
                layer_idx = int(name.split(".")[2])
                if "c_fc" in name:
                    self.base_model.model.layers[layer_idx].mlp.c_fc = svf_layer
                elif "c_proj" in name:
                    self.base_model.model.layers[layer_idx].mlp.c_proj = svf_layer

        initial_embedding = torch.zeros(2048).to(self.device)
        initial_z_vectors = [layer.z.clone().detach() for layer in self.svf_layers]
        self.nodes = [Node(initial_embedding, initial_z_vectors)]
        self.buffer = []
        self.distance_threshold = distance_threshold
        self.buffer_size = buffer_size
        self.temperature = temperature
        self.top_k = top_k
        self.lr = lr
        self.scaler = MinMaxScaler()

    def embed_input(self, text, params=None):
        if params:
            params_array = np.array(list(params.values())).reshape(1, -1)
            normalized_params = torch.tensor(self.scaler.fit_transform(params_array), device=self.device)
            weights = {'wear': 0.5, 'exhaust_temp': 0.3, 'vibration': 0.2}
            weighted_params = normalized_params * torch.tensor([weights.get(k, 0.0) for k in params.keys()], device=self.device)
            text += f" [PARAMS: {weighted_params}]"
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
        with torch.no_grad():
            outputs = self.base_model(**inputs, output_hidden_states=True)
            hidden_states = outputs.hidden_states[-1]
            return torch.mean(hidden_states, dim=1).squeeze(0)

    def compute_distance(self, emb1, emb2):
        return 1 - F.cosine_similarity(emb1.unsqueeze(0), emb2.unsqueeze(0)).item()

    def set_z_vectors(self, z_vectors):
        for layer, z in zip(self.svf_layers, z_vectors):
            layer.z.data = z.clone().to(self.device)

    def process_input(self, text, params=None, feedback=None):
        embedding = self.embed_input(text, params)
        distances = [self.compute_distance(embedding, node.embedding) for node in self.nodes]
        min_distance = min(distances)
        closest_idx = np.argmin(distances)
        closest_node = self.nodes[closest_idx]

        if min_distance > self.distance_threshold:
            new_z_vectors = [layer.z.clone().detach() for layer in self.svf_layers]
            new_node = Node(embedding, new_z_vectors)
            self.nodes.append(new_node)
            logging.info(f"New node created, total nodes: {len(self.nodes)}")
            closest_node = new_node
        else:
            closest_node.update_embedding(embedding)
            logging.info(f"Node {id(closest_node)} updated, embedding dist: {min_distance}")

        self.set_z_vectors(closest_node.z_vectors)
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
        outputs = self.base_model.generate(
            **inputs,
            max_new_tokens=50,
            num_return_sequences=3,  # Reduced to 3 for memory efficiency
            do_sample=True,
            temperature=self.temperature,
            top_k=self.top_k,
            pad_token_id=self.tokenizer.eos_token_id
        )
        responses = [self.tokenizer.decode(out[inputs.input_ids.shape[1]:], skip_special_tokens=True).strip() for out in outputs]
        response = max(set(responses), key=responses.count)  # Majority vote

        while response in closest_node.past_responses and feedback == -2:
            outputs = self.base_model.generate(
                **inputs,
                max_new_tokens=50,
                num_return_sequences=3,
                do_sample=True,
                temperature=self.temperature * 1.2,
                top_k=self.top_k,
                pad_token_id=self.tokenizer.eos_token_id
            )
            responses = [self.tokenizer.decode(out[inputs.input_ids.shape[1]:], skip_special_tokens=True).strip() for out in outputs]
            response = max(set(responses), key=responses.count)

        reward = 1  # Default positive
        if params and 'wear' in params:
            wear = float(params['wear'])
            if wear > 3.0 and 'replace' not in response.lower():
                reward = -2
            elif 1.5 <= wear <= 3.0 and 'maintenance' not in response.lower():
                reward = -2
            elif wear < 1.5 and 'check' not in response.lower():
                reward = -2
            elif feedback is not None:
                reward = feedback

        closest_node.past_responses.add(response)
        self.buffer.append((text, response, reward, closest_node))
        logging.info(f"Generated response: {response}, Reward: {reward}, Nodes: {len(self.nodes)}")

        if len(self.buffer) >= self.buffer_size:
            self.update_nodes()

        return response

    def update_nodes(self):
        node_data = defaultdict(list)
        for text, response, reward, node in self.buffer:
            node_data[id(node)].append((text, response, reward))

        for node in self.nodes:
            data = node_data.get(id(node), [])
            if data:
                optimizers = [torch.optim.Adam([z], lr=self.lr) for z in node.z_vectors]
                for text, response, reward in data:
                    full_text = text + " " + response
                    inputs = self.tokenizer(full_text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
                    input_len = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids.shape[1]
                    targets = self.tokenizer(response, return_tensors="pt", padding=True, truncation=True, max_length=50).input_ids.to(self.device)

                    self.set_z_vectors(node.z_vectors)
                    outputs = self.base_model(**inputs)
                    logits = outputs.logits[:, input_len-1:-1, :]
                    if logits.shape[1] < targets.shape[1]:
                        padding = torch.zeros((1, targets.shape[1] - logits.shape[1], logits.shape[2]), device=self.device)
                        logits = torch.cat([logits, padding], dim=1)
                    elif logits.shape[1] > targets.shape[1]:
                        logits = logits[:, :targets.shape[1], :]

                    log_probs = F.log_softmax(logits, dim=-1)
                    target_log_probs = log_probs.gather(2, targets.unsqueeze(-1)).squeeze(-1)
                    loss = -reward * target_log_probs.mean()
                    for opt in optimizers:
                        opt.zero_grad()
                    loss.backward()
                    for opt in optimizers:
                        opt.step()
                logging.info(f"Node {id(node)} updated with loss: {loss.item()}")
                torch.cuda.empty_cache()  # Clear GPU memory after each node update
        self.buffer = []
        torch.cuda.empty_cache()  # Clear GPU memory after buffer processing

# Mock InfluxDB data (replace with actual integration)
def get_engine_data(engine_id):
    data = {
        "flight_hours": random.uniform(5000, 8000),
        "exhaust_temp": random.uniform(600, 750),
        "vibration": random.uniform(0.5, 12.5),
        "thrust": random.uniform(50, 75),
        "pressure": random.uniform(1.0, 13561),
        "wear": random.uniform(0.5, 4.0)
    }
    return data

# Function to evaluate on engine dataset
def evaluate_engine(model, num_engines=1326, batch_size=100):
    correct = 0
    total = 0
    for batch_start in range(0, num_engines, batch_size):
        batch_end = min(batch_start + batch_size, num_engines)
        for i in range(batch_start, batch_end):
            params = get_engine_data(i)
            wear = params['wear']
            prompt = (f"Given engine with flight_hours={params['flight_hours']:.2f}h, "
                      f"exhaust_temp={params['exhaust_temp']:.2f}°C, vibration={params['vibration']:.2f}g, "
                      f"thrust={params['thrust']:.2f}kN, pressure={params['pressure']:.2f}bar, "
                      f"wear={wear:.2f}, predict maintenance action: ‘replace’, ‘maintenance’, or ‘check’ "
                      f"based on wear thresholds: wear > 3.0 = replace, 1.5–3.0 = maintenance, <1.5 = check.")
            response = model.process_input(prompt, params)
            try:
                if wear > 3.0 and 'replace' in response.lower():
                    correct += 1
                elif 1.5 <= wear <= 3.0 and 'maintenance' in response.lower():
                    correct += 1
                elif wear < 1.5 and 'check' in response.lower():
                    correct += 1
                total += 1
                feedback = 1 if (wear > 3.0 and 'replace' in response.lower()) or \
                               (1.5 <= wear <= 3.0 and 'maintenance' in response.lower()) or \
                               (wear < 1.5 and 'check' in response.lower()) else -2
                model.process_input(prompt, params, feedback)
                logging.info(f"Engine {i}: Predicted {response}, True {get_true_action(wear)}, Correct: {feedback == 1}")
            except Exception as e:
                logging.error(f"Error processing engine {i}: {e}")
                total += 1
            torch.cuda.empty_cache()  # Clear GPU memory after each engine
        logging.info(f"Completed batch {batch_start}-{batch_end-1}")
    accuracy = correct / total * 100
    logging.info(f"Evaluation complete. Accuracy: {accuracy}% on {total} engines")
    with open('gesal_evaluation_results.txt', 'w') as f:
        f.write(f"Accuracy: {accuracy}%\nNodes: {len(model.nodes)}\n")
    logging.getLogger().handlers[0].flush()
    return accuracy

def get_true_action(wear):
    if wear > 3.0:
        return "replace"
    elif 1.5 <= wear <= 3.0:
        return "maintenance"
    else:
        return "check"

# Warm-up phase
def warm_up(model, num_warmup=5):
    print("Starting warm-up phase...")
    for i in range(num_warmup):
        params = get_engine_data(i)
        prompt = (f"Given engine with flight_hours={params['flight_hours']:.2f}h, "
                  f"exhaust_temp={params['exhaust_temp']:.2f}°C, vibration={params['vibration']:.2f}g, "
                  f"thrust={params['thrust']:.2f}kN, pressure={params['pressure']:.2f}bar, "
                  f"wear={params['wear']:.2f}, predict maintenance action: ‘replace’, ‘maintenance’, or ‘check’ "
                  f"based on wear thresholds: wear > 3.0 = replace, 1.5–3.0 = maintenance, <1.5 = check.")
        response = model.process_input(prompt, params)
        true_action = get_true_action(params['wear'])
        feedback = 1 if true_action in response.lower() else -2
        model.process_input(prompt, params, feedback)
        logging.info(f"Warm-up {i}: Predicted {response}, True {true_action}, Feedback: {feedback}")
    print("Warm-up complete. Beginning full evaluation...")

# Main execution
if __name__ == "__main__":
    # Initialize model
    model = AdaptiveLLM(
        distance_threshold=0.1,
        buffer_size=20,
        temperature=0.3,
        top_k=20,
        lr=0.005
    )

    # Warm-up phase
    warm_up(model)

    # Run full evaluation
    print("Starting GESAL evaluation on engine dataset...")
    accuracy = evaluate_engine(model, batch_size=100)
    print(f"Final Accuracy on Engine Dataset: {accuracy}%")
    print(f"Total Nodes Created: {len(model.nodes)}")

    # Display logs and results
    try:
        with open('gesal_training_logs.txt', 'r') as f:
            print("Logs:")
            print(f.read())
        with open('gesal_evaluation_results.txt', 'r') as f:
            print("Results:")
            print(f.read())
    except FileNotFoundError:
        print("Warning: Log or results file not found. Check for errors above.")

Starting warm-up phase...
Warm-up complete. Beginning full evaluation...
Starting GESAL evaluation on engine dataset...
Final Accuracy on Engine Dataset: 40.79939668174962%
Total Nodes Created: 2
